# Sensor Confirmation
In this notebook the data exploration for Sensor Confirmation will be presented. Assumptions about sensor pairs will be explained and visuals to support certain actions will be provided, i.e., offset and scaling of sensor readings. The goal is to determine how feasible Sensor Confirmation is given data readings from a set of independent sensors that resolve to the same information.

-  Sensors Explored
    -  Inertial Measurement Unit (IMU)
        -  Accelerometer
        -  Gyroscope
        -  Magnetometer
    -  Optical Flow (OF)
    -  Rangefinder (RF)
    -  Global Positioning System (GPS)
    -  Barometer
***
| Sensor Systems                                              | Gyroscope | Accelerometer | Magnetometer | Barometer | GPS   | Rangefinder | Optical Flow | Rangefinder & Optical Flow | Rangefinder & Optical Flow & <br>(Gyroscope \|\|  Magnetometer) | Accelerometer & <br>(Gyroscope \|\|  Magnetometer) |
|-------------------------------------------------------------|-----------|---------------|--------------|-----------|-------|-------------|--------------|----------------------------|-------------------------------------------------------------|------------------------------------------------|
| Gyroscope                                                   |     -     |               |     **x**    |           |       |             |     **x**    |                            |                              -                              |                        -                       |
| Accelerometer                                               |     -     |       -       |              |   **x**   |       |    **x**    |              |            **x**           |                            **x**                            |                        -                       |
| Magnetometer                                                |     -     |       -       |      -       |           |       |             |     **x**    |                            |                              -                              |                        -                       |
| Barometer                                                   |     -     |       -       |       -      |      -    | **x** |    **x**    |              |            **x**           |                            **x**                            |                      **x**                     |
| GPS                                                         |     -     |       -       |       -      |     -     |   -   |    **x**    |              |                            |                            **x**                            |                      **x**                     |
| Rangefinder                                                 |     -     |       -       |       -      |     -     |   -   |      -      |              |              -             |                              -                              |                      **x**                     |
| Optical Flow                                                |     -     |       -       |       -      |     -     |   -   |      -      |       -      |              -             |                              -                              |                      **x**                     |
| Rangefinder & Optical Flow                                  |     -     |       -       |       -      |     -     |   -   |      -      |       -      |              -             |                              -                              |                      **x**                     |
| Rangefinder & Optical Flow & <br>(Gyroscope \|\|  Magnetometer) |     -     |       -       |       -      |     -     |   -   |      -      |       -      |              -             |                              -                              |                      **x**                     |
| Accelerometer & <br>(Gyroscope \|\|  Magnetometer)              |     -     |       -       |       -      |     -     |   -   |      -      |       -      |             -              |                              -                              |                        -                       |

In [1]:
%load_ext autoreload
%autoreload 2

## Data Processing
The log `file` needs to be parsed out as the format is irregular, starting with
a csv of rows prefixed with FMT, indicating the log type and format. The FMT
rows will be parsed into headers and then the subsequent rows will be matched
on log name, i.e., "__SNS1__", "__SNS2__", "__SNS3__" and added to their
respective dataframes. The dataframes are then unioned on their timestamps to
create a single dataframe with all of the __SNS__ data. If an `output` path is
provided the newly created dataframe will be written to the path as a CSV.

In [2]:
import pandas as pd
import numpy as np
from confirmation.process import process_SNS
from os.path import exists

#Convert log file from Mission Planner to usable CSV
data_dir = "../data/"
file = "C-Delivery.log" #Change file for different log files
output = data_dir + "interim/" + file[:-3] + "csv"

#Check if file exists already and if it does skip processing and read the file
if not exists(output):
    #Open log file and parse out the SNS data into pandas dataframes
    df = process_SNS(data_dir + "raw/" + file, output)
else:
    df = pd.read_csv(output)

  0%|          | 0/511825 [00:00<?, ?it/s]

## Gyroscope and Magnetometer
Gyroscopes and Magnetometer are both related in that they provide attitude
information. Gyroscopes provide angular rate information while Magnetometers
provide magnetic field information which can be used with local corrections
to determine heading.

The gyroscopes we generally use are the ones embedded in the
ArduPilot flight controllers which are microelectromechanical systems
(MEMS) gyroscopes. MEMS gyroscopes measure angular rate using the Coriolis
effect. The angular rate by itself can be used as change in attitude but
can also see use in systems such as dead reckoning where a starting attitude
is known and the angular rate is integrated over time to rotate the attitude.
A known issue with dead reckoning however is integration drift, where the
attitude will drift from the true attitude due to integrating noise in the
angular rate readings.

The magnetometer is used for absolute attitude information. By measuring
the magnetic field and compensating for local magnetic declination an
attitude can be established. ArduPilot primarily uses the magnetometer for
heading correction which represents the yaw. Several yaw measurements
can be used to determine the angular rate of the system which can be
confirmed directly to the gyroscope angular rate measurements.

The two sensors above represent one of the more elementary sensor pairs that
can be confirmed with one another without augmentation from other sensors.

#### Gyroscope
Gyroscope inherently provides angular rate which can be confirmed to the 
changes in magnetometer-derived change in attitude. If we were to confirm
directly to the magnetometer-derived attitude instead we would need to
integrate the gyroscope readings.

Here is a plot of the gyroscope readings along the x axis during the duration
of the flight, representing the pitching of the quadcopter.

In [ ]:
#TODO: Data exploration for Gyroscope vs Mag
from confirmation.visualize import simple_time_plot

simple_time_plot(df.TimeUS, df.)